In [ ]:
# default_exp metrics

# Custom losses and metrics

In [ ]:
# export
from drone_detector.imports import *
from fastai.learner import Metric
from fastai.torch_core import *
from fastai.metrics import *
from fastai.losses import BaseLoss
import sklearn.metrics as skm
import torch
import torch.nn.functional as F

In [ ]:
# export
mk_class('ActivationType', **{o:o.lower() for o in ['No', 'Sigmoid', 'Softmax', 'BinarySoftmax']},
         doc="All possible activation classes for `AccumMetric")

In [ ]:
#export

def adjusted_R2Score(r2_score, n, k):
    "Calculates adjusted_R2Score based on r2_score, number of observations (n) and number of predictor variables(k)"
    return 1 - (((n-1)/(n-k-1)) * (1 - r2_score))


In [ ]:
#export

def _rrmse(inp, targ):
    "RMSE normalized with mean of the target"
    return torch.sqrt(F.mse_loss(inp, targ)) / targ.mean() * 100

rrmse = AccumMetric(_rrmse)
rrmse.__doc__ = "Target mean weighted rmse"

In [ ]:
#export
def _bias(inp, targ):
    "Average bias of predictions"
    inp, targ = flatten_check(inp, targ)
    return (inp - targ).sum() / len(targ)

bias = AccumMetric(_bias)
bias.__doc__ = "Average bias of predictions"

In [ ]:
#export
def _bias_pct(inp, targ):
    "Mean weighted bias"
    inp, targ = flatten_check(inp, targ)
    return 100 * ((inp-targ).sum()/len(targ)) / targ.mean()

bias_pct = AccumMetric(_bias_pct)
bias_pct.__doc__ = 'Mean weighted bias'

BigEarthNet metrics

In [ ]:
#export

def label_ranking_average_precision_score(sigmoid=True, sample_weight=None):
    """Label ranking average precision (LRAP) is the average over each ground truth label assigned to each sample, 
    of the ratio of true vs. total labels with lower score."""
    activation = ActivationType.Sigmoid if sigmoid else ActivationType.No
    return skm_to_fastai(skm.label_ranking_average_precision_score, sample_weight=None, flatten=False, thresh=None, 
                         activation=activation)

In [ ]:
# export

def label_ranking_loss(sigmoid=True, sample_weight=None):
    """Compute the average number of label pairs that are incorrectly ordered given y_score 
    weighted by the size of the label set and the number of labels not in the label set."""
    activation = ActivationType.Sigmoid if sigmoid else ActivationType.No
    return skm_to_fastai(skm.label_ranking_loss, sample_weight=None, flatten=False, thresh=None, 
                         activation=activation)

In [ ]:
# export

def _one_error(inp, targ):
    max_ranks = inp.argmax(axis=1)
    faults = 0
    for i in range_of(max_ranks):
        faults += targ[i,max_ranks[i]]
    return 1 - torch.true_divide(faults, len(max_ranks))
    
one_error = AccumMetric(_one_error, flatten=False)
one_error.__doc__ = "Rate for which the top ranked label is not among ground truth"

In [ ]:
# export

def coverage_error(sigmoid=True, sample_weight=None):
    """Compute how far we need to go through the ranked scores to cover all true labels. 
    The best value is equal to the average number of labels in y_true per sample."""
    
    activation = ActivationType.Sigmoid if sigmoid else ActivationType.No
    return skm_to_fastai(skm.coverage_error, sample_weight=None, flatten=False, thresh=None, activation=activation)

In [ ]:
from fastai.learner import Learner
class TstLearner(Learner):
    def __init__(self,dls=None,model=None,**kwargs): self.pred,self.xb,self.yb = None,None,None

def compute_val(met, x1, x2):
    met.reset()
    vals = [0,6,15,20]
    learn = TstLearner()
    for i in range(3):
        learn.pred,learn.yb = x1[vals[i]:vals[i+1]],(x2[vals[i]:vals[i+1]],)
        met.accumulate(learn)
    return met.value

In [ ]:
lrap = label_ranking_average_precision_score()
lrl = label_ranking_loss()
cov = coverage_error()

In [ ]:
x_1 = torch.randn(10,10)
x_2 = torch.randint(2,(10,10))
x_1, torch.sigmoid(x_1), x_2

(tensor([[-0.9863,  0.5349,  0.1619,  1.5855,  1.4731, -1.7488, -0.7248,  1.4115,
           0.2501,  0.0907],
         [ 0.6307, -2.8401,  1.0606,  1.4899,  0.2854, -0.2732, -0.5758, -0.4963,
          -2.2258,  1.9919],
         [ 0.2674, -0.0408,  0.1790, -1.1830, -0.1299, -1.3078,  0.4488,  0.4553,
           0.5611, -1.1726],
         [ 1.3335, -0.2782,  0.8992, -0.5221,  0.6512, -0.2832,  0.7030,  0.3703,
           0.2219,  1.1805],
         [ 1.4847,  0.5350, -0.6607,  0.6501,  1.0741, -0.7880,  0.3460, -0.1751,
           1.0657,  0.2154],
         [-0.6447, -0.2532, -0.1455,  0.2927,  0.8283,  0.9487, -1.6784, -0.1991,
           0.8065, -1.1441],
         [-0.6307, -1.0541, -0.1082, -1.1037, -1.3668, -0.3828, -1.1724,  0.6949,
           1.3225,  0.4474],
         [-0.0389,  0.0222, -2.2714, -0.1187,  0.5216,  0.8554,  0.0150, -1.5269,
          -1.1638,  0.3692],
         [-0.0507, -1.6455,  0.2197, -0.0557, -0.6117, -1.7797, -0.6482, -0.7175,
           2.2846,  0.2458],
 

In [ ]:
compute_val(lrl, x_1, x_2)

0.49940476190476196

In [ ]:
compute_val(lrap, x_1, x_2)

0.5268088624338624

In [ ]:
compute_val(cov, x_1, x_2)

8.4

In [ ]:
one_error(x_1, x_2)

tensor(0.6000)